In [ ]:
import numpy as np
import soundfile as sf
import sounddevice as sd
from scipy.signal import fftconvolve
import tools

In [ ]:
fs = 44100
stddev = 0.2
snr = -21

speech, fs = sf.read('data/xmas.wav')
noise = np.random.normal(scale=stddev, size=(len(speech)))
speech *= np.linalg.norm(noise) / np.linalg.norm(speech) * 10**(snr / 20) 

hrir_center, _ = sf.read('data/hrir00.wav')
hrir_lateral, _ = sf.read('data/hrir90.wav')

ear_speech_center = np.column_stack([fftconvolve(h, speech) for h in hrir_center.T])
ear_noise_center = np.column_stack([fftconvolve(h, noise) for h in hrir_center.T])
ear_noise_lateral = np.column_stack([fftconvolve(h, noise) for h in hrir_lateral.T])

sd.play(ear_speech_center + ear_noise_center, fs)
sd.wait()
sd.play(ear_speech_center + ear_noise_lateral, fs)
sd.wait()

In [ ]:
def delay_and_attenuate(x, delay, gain, fs=44100):
    """
    Parameters
    ----------
    x : array_like
        Mono signal
    delay : float
        Delay of the second channel (in milliseconds)
    gain : float
        Gain of the second channel (in dB)
    """
    d = int(np.round(delay * fs * 0.001))
    x = np.tile(np.concatenate((x, np.zeros(np.abs(d))), axis=-1), (2, 1)).T
    x[:, 1] = 10**(gain / 20) * np.roll(x[:, 1], d, axis=0)
        
    return tools.normalize(x, maximum=0.6)

In [ ]:
delay = 3
gain = -10
s, fs = sf.read('data/xmas.wav')

x = delay_and_attenuate(s, delay, gain)

sd.play(x, fs)
sd.wait()

In [ ]:
def virtual_stereo(x, **kwargs):
    """
    
    """
    # load HRIRs
    hright, _ = sf.read('data/hrir30.wav')
    hleft = np.roll(hright, 1)

    return np.column_stack([fftconvolve(x[:, 0], ir, **kwargs) for ir in hleft.T]) \
            + np.column_stack([fftconvolve(x[:, 1], ir, **kwargs) for ir in hright.T])

In [ ]:
delay = 0
gain = 0
mono_signal, _ = sf.read('data/xmas.wav')

stereo_signal = delay_and_attenuate(mono_signal, delay, gain)
y = tools.normalize(virtual_stereo(stereo_signal), maximum=0.6)

sd.play(y)
sd.wait()

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>